In [1]:
import awkward as ak
import numpy as np
import time
# print(coffea.__version__)
from coffea import util
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
import mplhep as hep
import matplotlib.pyplot as plt

plt.style.use([hep.style.CMS, hep.style.firamath])
import hist
from hist.intervals import ratio_uncertainty
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

from python.plugins import runCoffeaJob
from python.triggerProcessor import triggerProcessor, applyPrescales
from dask.distributed import Client

In [3]:
# in_year = '2016APV'
# data_bool = True
# processor = triggerProcessor(year = in_year, trigger = 'PFJet', data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "fileset_JetHT_wRedirs.json"
#     # filename = "datasets_JetHT_NOAK8PFHLT.json"
# result = runCoffeaJob(processor, jsonFile = filename, casa = False, dask = True, testing = False, year = processor.year, data = processor.data)
# with open('coffeaOutput/triggerAssignmentL1pt_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)

In [4]:
# in_year = '2016'
# data_bool = True
# processor = triggerProcessor(year = in_year, trigger = 'PFJet', data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "fileset_JetHT_wRedirs.json"
#     # filename = "datasets_JetHT_NOAK8PFHLT.json"
# result = runCoffeaJob(processor, jsonFile = filename, casa = False, dask = True, testing = False, year = processor.year, data = processor.data)
# with open('coffeaOutput/triggerAssignmentL1pt_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)

In [5]:
# in_year = '2017'
# data_bool = True
# processor = triggerProcessor(year = in_year, trigger = 'PFJet', data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "fileset_JetHT_wRedirs.json"
#     # filename = "datasets_JetHT_NOAK8PFHLT.json"
# result = runCoffeaJob(processor, jsonFile = filename, casa = False, dask = True, testing = False, year = processor.year, data = processor.data)
# with open('coffeaOutput/triggerAssignmentL1pt_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)

In [6]:
# in_year = '2018'
# data_bool = True
# processor = triggerProcessor(year = in_year, trigger = 'PFJet', data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "fileset_JetHT_wRedirs.json"
#     # filename = "datasets_JetHT_NOAK8PFHLT.json"
# result = runCoffeaJob(processor, jsonFile = filename, casa = False, dask = True, testing =False, year = processor.year, data = processor.data)
# with open('coffeaOutput/triggerAssignmentL1pt_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)

In [7]:
from scipy.optimize import minimize, curve_fit
from scipy.stats import rv_continuous, norm
from scipy.special import erf, erfinv

def fit_function(x, b, c):
    # y = a+0.5*(1-a)*(1+erf((x-b)/c))
    y = 0.5*(1+erf((x-b)/(np.sqrt(2)*c)))
    return y
#### define custom error function using scipy.stats rv_continuous --> will automatically inherit MLE fit

class cust_erf(rv_continuous):
    def _pdf(self, x, b, c):
        # return a+0.5*(1-a)*(1+erf((x-b)/c))
        return 0.5*(1+erf((x-b)/(np.sqrt(2)*c)))
    #### use rv_continuous scale and loc to compute x->(x-loc)/scale
fit_func = cust_erf(name='erf')
print("rv_cont fit function: ", fit_func)

#### params are b,c
                      
                      
def fit_trigEff(x, data):
    #x is the pt bin centers; data is the efficiency values
    x0 = [np.min(x), np.min(x)/5]
    popt, pcov = curve_fit(fit_function, x, data, p0 = x0)
    # b, c, loc, scale = fit_func.fit(data, *popt, floc = 0., fscale = 1.0)
    print("Basic fit results: ", popt)
    return popt

from python.plugins import checkdir
def plot_turnOn(result, HLT, label):
    hist_trigEff = result['hist_trigEff_ptCut']
    hist_trigRef = result['hist_trigRef']
    hist_pt = result['hist_pt']
    HLT_paths = [path for path in list(hist_trigEff.project("HLT_cat").axes[0])]
    dataset = hist_trigEff[{"HLT_cat":slice(0,hist.overflow-1,sum)}].axes[0][0]
    os_path = 'plots/triggerStudies'+ dataset+'/'
    checkdir(os_path)
    print('HLT_paths:', HLT_paths)
    trigThresh = [int(path.replace(HLT, '')) for path in HLT_paths]
    turnOnPts = {}
    turnOnPts_ptCut = {}
    for i in np.arange(len(HLT_paths)):
        path = HLT_paths[i]
        #print("Path = ", path)
        #print("dataset: ", dataset)
        # print("trigeffvalues for path: ", hist_trigEff[{"HLT_cat":path}])
        #### numerator is number of events that belong to trigger path below and have pt > current trigger thresh
        numerator = hist_trigEff[{"HLT_cat":path}].values()[0]
        #### denominator is number of events in trigger below
        denominator = hist_trigRef[{"HLT_cat":path}].values()[0]
        num = numerator[denominator > 0]
        denom = denominator[denominator > 0]
        efficiency = num / denom
        # eff_hist = np.where(denominator > 0, numerator/denominator, 0)
        eff_hist = np.divide(
                hist_trigEff[{"HLT_cat":path}].values(),
                hist_trigRef[{"HLT_cat":path}].values(),
                out=np.empty(np.array(hist_trigRef[{"HLT_cat":path}].values()).shape).fill(np.nan),
                where=hist_trigRef[{"HLT_cat":path}].values()!= 0)[0]
        err_up, err_dn = ratio_uncertainty(hist_trigEff[{"HLT_cat":path}].values(), hist_trigRef[{"HLT_cat":path}].values(), 'efficiency')
        # print(err_up[0], err_dn[0])
        eff_hist_err = np.array([np.divide(
                np.sqrt(hist_trigEff[{"HLT_cat":path}].values())/2,
                hist_trigRef[{"HLT_cat":path}].values(),
                out=np.empty(np.array(hist_trigRef[{"HLT_cat":path}].values()).shape).fill(np.nan),
                where=hist_trigRef[{"HLT_cat":path}].values()!= 0)[0], np.zeros_like(hist_trigRef[{"HLT_cat":path}].values())[0]])
        pt_centers = hist_trigEff.axes['pt'].centers[denominator > 0]
        # print("Pt centers: ", pt_centers)
        # print("Pt edges: ",  hist_trigEff.axes['pt'].edges)
        #### plot original numerator and denominator histograms
        fig, ax = plt.subplots(1, 1)
        hist_trigEff[{"HLT_cat":path}].plot1d(ax=ax)
        hist_trigRef[{"HLT_cat":path}].plot1d(ax=ax)
        ax.set_title("Events in HLT " + HLT_paths[i-1])
        ax.set_xlabel("Leading Jet pT (GeV)")
        ax.set_ylim(1, None)
        ax.legend(["Numerator", "Numerator_ptCut", "Denominator"])
        plt.savefig(os_path + 'numDenom_HLT_' + path + ".png")
        #### Make custom fit function for the efficiencies
        if len(efficiency) > 0:
            fig, ax = plt.subplots(1, 1)
            ax.set_ylim([-0.01, 1.1])
            ### only fit points after trigger threshold
            eff_turnon_id = np.min(np.argwhere(trigThresh[i] <= pt_centers))
            if 4 > i >= 2:
                xdata = pt_centers[eff_turnon_id+i-1:]
                ydata = efficiency[eff_turnon_id+i-1:]
            elif 6 > i>= 4:
                xdata = pt_centers[eff_turnon_id+i-2:]
                ydata = efficiency[eff_turnon_id+i-2:]
            elif i>= 6:
                xdata = pt_centers[eff_turnon_id+3:]
                ydata = efficiency[eff_turnon_id+3:]
            else:
                xdata = pt_centers[eff_turnon_id:]
                ydata = efficiency[eff_turnon_id:]
            popt = fit_trigEff(xdata, ydata)
            # b_mle, c_mle, popt = fit_trigEff(pt_centers, efficiency)
            xpts = np.linspace(pt_centers[eff_turnon_id], 2400, int((2400-pt_centers[eff_turnon_id])))
            ### turn on point is pt value when efficiency reaches 0.99 --> solve fitted equation
            b, c = popt
            xdata = pt_centers[eff_turnon_id:]
            ydata = efficiency[eff_turnon_id:]
            def NLL(params):
                b = params[0]
                c = params[1]
                y = 0.5*(1+erf((xdata-b)/(np.sqrt(2)*c)))
                return -1*norm(y, params[2]).logpdf(ydata).sum()
            # res = minimize(NLL, [popt[0], popt[1], 1])
            res = minimize(NLL, [np.min(xdata), np.min(xdata)/5, 1])
            b_mle, c_mle, s_mle = res.x
            print("MLE results ", res.x)
            to_pt = (np.sqrt(2)*c*erfinv(2*0.99-1)) + b
            to_pt_mle = (np.sqrt(2)*c_mle*erfinv(2*0.99-1)) + b_mle
            turnOnPts[path] = [to_pt, to_pt_mle]
            hep.histplot(eff_hist, hist_trigEff.axes['pt'].edges, ax = ax, label='HLT_'+path, yerr=np.array([err_up[0],err_dn[0]]), histtype='errorbar', color = 'black', marker =["o"], markersize=6,)
            ax.plot(xpts, fit_function(xpts, *popt), label = "Fit; to_pt = %.f"%to_pt, color='red')
            hep.cms.label("Preliminary", lumi = label, data = True, loc=0, ax=ax, fontsize=18);
            #ax.plot(xpts, fit_function(xpts, b_mle, c_mle), label = "MLE fit; to_pt = %.f"%to_pt_mle)
            ax.set_xlabel("Leading Jet pT (GeV)")
            ax.legend()
            plt.savefig(os_path + 'efficiency_HLT_'+path + ".png")
    #### plot total leading jet pt for reference
    fig, ax = plt.subplots(1, 1)
#     hist_pt.plot1d(ax=ax)
    pt_hist = hist_trigEff[{"dataset":sum}]
    pt_hist.plot1d(ax=ax, overlay="HLT_cat", stack = True)
    ax.set_title("All events")
    ax.set_xlabel("Leading Jet pT (GeV)")
    ax.set_ylim(1, None)
    ax.set_xlim([180., 3200.])
    plt.legend()
    plt.savefig(os_path + 'HLT_' + path + ".png")
    
    return turnOnPts


# lumi = {'2018' : 59.74,
#         '2017': 41.48,
#         '2016': 36.33 
#        }

#'2016APV':19.5
#"2016":16.8

rv_cont fit function:  <__main__.cust_erf object at 0x7fd74bd9b340>


In [8]:
# HLT = 'PFJet'
# with open('coffeaOutput/triggerAssignmentL1pt_JetHT_2016APV_PFJet.pkl', "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT, label=r'2016 APV - 19.5' )
# print(turnOnPts)

In [9]:
# HLT = 'PFJet'
# with open('coffeaOutput/triggerAssignmentL1pt_JetHT_2017_PFJet.pkl', "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT, label=r'2017 - 41.48', )
# print(turnOnPts)

In [10]:
# HLT = 'PFJet'
# with open('coffeaOutput/triggerAssignmentL1pt_JetHT_2018_PFJet.pkl', "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT, label=r'2018 - 59.74', )
# print(turnOnPts)

In [11]:
# HLT = 'PFJet'
# with open('coffeaOutput/triggerAssignmentL1pt_JetHT_2016_PFJet.pkl', "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT, label=r'Late 2016 - 16.8' )
# print(turnOnPts)

In [12]:
# HLT = 'AK8PFJet'
# with open("coffeaOutput/triggerAssignmentL1pt_QCDsim_2017_PFJet.pkl", "rb") as f:
#     result = pickle.load( f )
# turnOnPts = plot_turnOn(result, HLT, label=r'2017 - 41.48')
# print(turnOnPts)

In [13]:
# HLT = 'AK8PFJet'
# result = util.load('coffeaOutput/triggerAssignment_QCDsim_2018_AK8PFJet_NewHist.coffea')
# turnOnPts = plot_turnOn(result, HLT, label=r'2018 - 59.74')
# print(turnOnPts)

In [14]:
newl12016APV = {'AK8PFJet60': [139.5571999490993, 245.53535229795398], 'AK8PFJet80': [208.43956364614138, 245.8326001605219], 
                 'AK8PFJet140': [290.39564432515516, 246.01436032449925], 'AK8PFJet200': [386.9131362346668, 316.5993928376519], 
                 'AK8PFJet260': [458.155057383878, 386.9622490345946], 'AK8PFJet320': [550.9902289618454, 527.4372732800409], 
                 'AK8PFJet400': [648.8680463439613, 597.869306052639], 'AK8PFJet450': [733.6385194424329, 668.2051368021362], 
                 'AK8PFJet500': [821.4588783253023, 738.5446622605812]}
newl12016 = {'AK8PFJet60': [139.5571999490993, 245.53535229795398], 'AK8PFJet80': [208.43956364614138, 245.8326001605219], 
             'AK8PFJet140': [290.39564432515516, 246.01436032449925], 'AK8PFJet200': [386.9131362346668, 316.5993928376519], 
             'AK8PFJet260': [458.155057383878, 386.9622490345946], 'AK8PFJet320': [550.9902289618454, 527.4372732800409], 
             'AK8PFJet400': [648.8680463439613, 597.869306052639], 'AK8PFJet450': [733.6385194424329, 668.2051368021362], 
             'AK8PFJet500': [821.4588783253023, 738.5446622605812]}
newl12017 = {'AK8PFJet80': [160.52022125699813, 245.85202220965644], 'AK8PFJet140': [277.85271111436055, 246.10402869403651], 
             'AK8PFJet200': [318.6411767123043, 316.6185299473434], 'AK8PFJet260': [459.1656517376684, 386.9894328364755], 
             'AK8PFJet320': [562.0252237627351, 527.4792214934412], 'AK8PFJet400': [643.851069453488, 597.9284546318087], 
             'AK8PFJet450': [705.0788178200379, 668.2475354949781], 'AK8PFJet500': [762.7485185854354, 738.5812156526756], 
             'AK8PFJet550': [815.002094554022, 808.9197189538069], 'AK8PFJet60': [205.05400523227544, 245.684911515468]}
newl12018 = {'AK8PFJet60': [-332.1769145662199, 245.650955165529], 'AK8PFJet80': [160.89471879512197, 245.86440968922605], 
             'AK8PFJet140': [274.1725162164536, 246.08377938512882], 'AK8PFJet200': [395.60974887544114, 316.5756365655121], 
             'AK8PFJet260': [475.25798095173286, 386.9816221068269], 'AK8PFJet320': [571.680946693499, 527.4729507490305], 
             'AK8PFJet400': [655.696422385234, 597.913991324915], 'AK8PFJet450': [711.5656697563014, 668.232929496411], 
             'AK8PFJet500': [768.1579923535201, 738.5606165982504], 'AK8PFJet550': [822.2737265359849, 808.8892513751434]}
pfjet2016APV = {'PFJet80': [221.50548457646764, 245.80359414733596], 'PFJet140': [297.303773316647, 246.05359519294237], 'PFJet200': [365.1336120673918, 316.5659648509295], 'PFJet260': [460.0898921606057, 386.9210814134716], 'PFJet320': [553.8485712694877, 527.4226587472542], 'PFJet400': [692.1212907064324, 597.8446126797236], 'PFJet450': [786.8856742383937, 668.1641235688161], 'PFJet500': [979.671786337144, 738.5027375039972], 'PFJet60': [123.93335136753808, 245.72432425034089]}
pfjet2017 = {'PFJet60': [-1595.2605509849727, 245.8172950616106], 'PFJet80': [141.2310007504739, 245.87476413219088], 'PFJet140': [280.9091493524036, 246.1037096100053], 'PFJet200': [382.1752391930248, 316.6018603013339], 'PFJet260': [456.6945321808703, 386.9753107466026], 'PFJet320': [559.0924890473514, 527.4627570990666], 'PFJet400': [635.2886366274465, 597.9123835576901], 'PFJet450': [720.0475114990843, 668.218477899657], 'PFJet500': [784.9475652849118, 738.5470644051876], 'PFJet550': [840.1791041568115, 808.8847580928046]}
pfjet2018 = {'PFJet60': [-120.82126509970408, 245.73957829941662], 'PFJet80': [160.72455293363703, 245.9068171283308], 'PFJet140': [287.63426567944975, 246.0911456568046], 'PFJet200': [395.64131492903664, 316.5666543646143], 'PFJet260': [474.98390359029395, 386.9655756313947], 'PFJet320': [571.4031774653945, 527.4573398293147], 'PFJet400': [647.1060350009018, 597.8975716942707], 'PFJet450': [721.9987922839626, 668.2018280982585], 'PFJet500': [783.2202950207775, 738.5288621407296], 'PFJet550': [839.0788762823904, 808.8570322939919]}
PFJetHT2016APV = {'PFJet60': 0., 
                        'PFJet80': 220., 
                        'PFJet140': 297., 
                        'PFJet200': 365., 
                        'PFJet260': 460., 
                        'PFJet320': 553., 
                        'PFJet400': 692., 
                        'PFJet450': 719., 
                        'PFJet500': 806.}
PFJetHT2016 = {'PFJet60': 71., 
                'PFJet80': 210., 
                'PFJet140': 295., 
                'PFJet200': 385., 
                'PFJet260': 456.,
                'PFJet320': 547., 
                'PFJet400': 640., 
                'PFJet450': 707., 
                'PFJet500': 789.}
PFJetHT2017 = {'PFJet60': 0., 
                    'PFJet80': 141., 
                    'PFJet140': 281., 
                    'PFJet200': 382.,
                    'PFJet260': 456., 
                    'PFJet320': 559., 
                    'PFJet400': 635., 
                    'PFJet450': 720, 
                    'PFJet500': 784., 
                    'PFJet550': 840.}
PFJetHT2018 = {'PFJet60': 0., 
                    'PFJet80': 161., 
                    'PFJet140': 287., 
                    'PFJet200': 395., 
                    'PFJet260': 474., 
                    'PFJet320': 571., 
                    'PFJet400': 647., 
                    'PFJet450': 721., 
                    'PFJet500': 783., 
                    'PFJet550': 839.}

JetHT2016_L1 = {'AK8PFJet40':0.,
                 'AK8PFJet60': 140., 
                 'AK8PFJet80': 210., 
                 'AK8PFJet140': 290., 
                 'AK8PFJet200': 380., 
                 'AK8PFJet260': 450., 
                 'AK8PFJet320': 550., 
                 'AK8PFJet400': 640., 
                 'AK8PFJet450': 690., 
                 'AK8PFJet500': 820.}
JetHT2016APV_L1 = {'AK8PFJet40':0., 
                     'AK8PFJet60': 140, 
                     'AK8PFJet80': 210., 
                     'AK8PFJet140': 290., 
                     'AK8PFJet200': 380., 
                     'AK8PFJet260': 450., 
                     'AK8PFJet320': 550., 
                     'AK8PFJet400': 640., 
                     'AK8PFJet450': 730., 
                     'AK8PFJet500': 820.}
JetHT2017_L1 = {'AK8PFJet40':0.,
                 'AK8PFJet60': 0., 
                 'AK8PFJet80': 160., 
                 'AK8PFJet140': 270., 
                 'AK8PFJet200': 310., 
                 'AK8PFJet260': 450., 
                 'AK8PFJet320': 560., 
                 'AK8PFJet400': 640., 
                 'AK8PFJet450': 700., 
                 'AK8PFJet500': 760., 
                 'AK8PFJet550': 810.}
JetHT2018_L1 = {'AK8PFJet15': 0.,
                 'AK8PFJet25': 0.,
                 'AK8PFJet40': 0.,
                 'AK8PFJet60': 0.,
                 'AK8PFJet80': 160., 
                 'AK8PFJet140': 270., 
                 'AK8PFJet200': 390., 
                 'AK8PFJet260': 470., 
                 'AK8PFJet320': 570., 
                 'AK8PFJet400': 650., 
                 'AK8PFJet450': 710., 
                 'AK8PFJet500': 760., 
                 'AK8PFJet550': 820.}

JetHT2016_new = {'AK8PFJet40':0.,
                 'AK8PFJet60': 100., 
                 'AK8PFJet80': 200., 
                 'AK8PFJet140': 260., 
                 'AK8PFJet200': 350., 
                 'AK8PFJet260': 420., 
                 'AK8PFJet320': 530., 
                 'AK8PFJet400': 620., 
                 'AK8PFJet450': 690., 
                 'AK8PFJet500': 740.}
JetHT2016APV_new = {'AK8PFJet40':0., 
                     'AK8PFJet60': 0, 
                     'AK8PFJet80': 220., 
                     'AK8PFJet140': 270., 
                     'AK8PFJet200': 360., 
                     'AK8PFJet260': 440., 
                     'AK8PFJet320': 530., 
                     'AK8PFJet400': 630., 
                     'AK8PFJet450': 690., 
                     'AK8PFJet500': 750.}
JetHT2017_new = {'AK8PFJet40':0.,
                 'AK8PFJet60': 0., 
                 'AK8PFJet80': 200., 
                 'AK8PFJet140': 260., 
                 'AK8PFJet200': 340., 
                 'AK8PFJet260': 420., 
                 'AK8PFJet320': 530., 
                 'AK8PFJet400': 620., 
                 'AK8PFJet450': 680., 
                 'AK8PFJet500': 745., 
                 'AK8PFJet550': 807.}
JetHT2018_new = {'AK8PFJet15': 0.,
                 'AK8PFJet25': 0.,
                 'AK8PFJet40': 0.,
                 'AK8PFJet60': 0.,
                 'AK8PFJet80': 0., 
                 'AK8PFJet140': 260., 
                 'AK8PFJet200': 360., 
                 'AK8PFJet260': 440., 
                 'AK8PFJet320': 550., 
                 'AK8PFJet400': 630., 
                 'AK8PFJet450': 700., 
                 'AK8PFJet500': 750., 
                 'AK8PFJet550': 810.}

In [15]:
in_year = '2016APV'
data_bool = True
winterfell = False
processor = applyPrescales(year = in_year, trigger = 'PFJet', turnOnPts = np.array(list(PFJetHT2016APV.values())), data = data_bool)
datastring = "JetHT" if processor.data == True else "QCDsim"
if processor.data==False and winterfell:
    filename = "QCD_flat_files.json"
elif processor.data==False:
    filename = "fileset_QCD.json"
else:
    filename = "fileset_JetHT_wRedirs.json"
result = runCoffeaJob(processor, jsonFile = filename, dask=True, testing = False , year = processor.year, data = processor.data)
with open('coffeaOutput/applyPrescalesL1_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
    pickle.dump( result, f)

Running over datasets  dict_keys(['/JetHT/Run2016B-ver1_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD', '/JetHT/Run2016C-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD', '/JetHT/Run2016D-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD', '/JetHT/Run2016E-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD', '/JetHT/Run2016F-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD'])
Running on LPC Condor


RuntimeError: No pool nodes could be contacted

In [ ]:
in_year = '2016'
data_bool = True
winterfell = False
processor = applyPrescales(year = in_year, trigger = 'PFJet', turnOnPts = np.array(list(PFJetHT2016.values())), data = data_bool)
datastring = "JetHT" if processor.data == True else "QCDsim"
if processor.data==False and winterfell:
    filename = "QCD_flat_files.json"
elif processor.data==False:
    filename = "fileset_QCD.json"
else:
    filename = "fileset_JetHT_wRedirs.json"
result = runCoffeaJob(processor, jsonFile = filename, dask=True, testing = False , year = processor.year, data = processor.data)
with open('coffeaOutput/applyPrescales_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
    pickle.dump( result, f)

In [ ]:
in_year = '2017'
data_bool = True
winterfell = False
processor = applyPrescales(year = in_year, trigger = 'PFJet', turnOnPts = np.array(list(PFJetHT2017.values())), data = data_bool)
datastring = "JetHT" if processor.data == True else "QCDsim"
if processor.data==False and winterfell:
    filename = "QCD_flat_files.json"
elif processor.data==False:
    filename = "fileset_QCD.json"
else:
    filename = "fileset_JetHT_wRedirs.json"
result = runCoffeaJob(processor, jsonFile = filename, dask=True, testing = False , year = processor.year, data = processor.data)
with open('coffeaOutput/applyPrescales_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
    pickle.dump( result, f)

In [ ]:
in_year = '2018'
data_bool = True
winterfell = False
processor = applyPrescales(year = in_year, trigger = 'AK8PFJet', turnOnPts = np.array(list(PFJetHT2018.values())), data = data_bool)
datastring = "JetHT" if processor.data == True else "QCDsim"
if processor.data==False and winterfell:
    filename = "QCD_flat_files.json"
elif processor.data==False:
    filename = "fileset_QCD.json"
else:
    filename = "fileset_JetHT_wRedirs.json"
result = runCoffeaJob(processor, jsonFile = filename, dask=True, testing = False , year = processor.year, data = processor.data)
with open('coffeaOutput/applyPrescalesL1_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
    pickle.dump( result, f)

In [ ]:
# in_year = '2016APV'
# data_bool = True
# winterfell = True
# processor = applyPrescales(year = in_year, trigger = 'PFJet', turnOnPts = np.array(list(JetHT2016APV_ver2.values())), data = data_bool)
# datastring = "JetHT" if processor.data == True else "QCDsim"
# if processor.data==False and winterfell:
#     filename = "QCD_flat_files.json"
# elif processor.data==False:
#     filename = "fileset_QCD.json"
# else:
#     filename = "datasets_JetHT_NOAK8PFHLT.json"
# result = runCoffeaJob(processor, jsonFile = filename, dask=True, testing = False, year = processor.year, data = processor.data)
# with open('coffeaOutput/applyPrescales_{}_{}_{}.pkl'.format(datastring, processor.year, processor.trigger), "wb") as f:
#     pickle.dump( result, f)

In [ ]:
def plotPrescales(result, year, label):
    hist_pt = result['hist_pt'][{'dataset':sum}]
    hist_pt_vals = np.where(hist_pt.values()==0., np.empty(np.array(hist_pt.values()).shape).fill(np.nan), hist_pt.values())
    hist_pt_byHLTpath_wPS = result['hist_pt_byHLTpath_wPS'][{'dataset':sum}]
    hist_pt_wPS = result['hist_pt_wPS'][{'dataset':sum}]
    wPS_vals = np.where(hist_pt_wPS.values()==0., np.empty(np.array(hist_pt_wPS.values()).shape).fill(np.nan), hist_pt_wPS.values())
    byHLTpath_vals = np.where(hist_pt_byHLTpath_wPS.values()==0., np.empty(np.array(hist_pt_byHLTpath_wPS.values()).shape).fill(np.nan), hist_pt_byHLTpath_wPS
                            .values())
    plt.rcParams['figure.facecolor']='white'
    plt.rcParams["figure.figsize"] = (40,10)
    fig, axs = plt.subplots(1, 3)      
    print(hist_pt)
    # hist_pt.plot(ax = axs[0], overlay='HLT_cat')
    axs[0].set_yscale('log')
    axs[0].set_ylim([0.0, 1.0e7])
    axs[0].set_xlim([0.0, 2400])
    axs[0].set_title('Sorted by HLT w/o prescale applied', pad=32)
    hep.histplot(hist_pt_vals, hist_pt.axes['pt'].edges, ax = axs[0],histtype='errorbar')
    hep.cms.label("Preliminary", lumi = label, data = True, loc=2, ax=axs[1], fontsize=18);
    hep.histplot(wPS_vals, hist_pt_wPS.axes['pt'].edges, ax = axs[1],histtype='errorbar')
    axs[1].set_yscale('log')
    axs[1].set_ylim([0.0, 1.0e10])
    axs[1].set_xlim([0.0, 2400])
    axs[1].set_title('Sorted by HLT w/ prescale applied', pad=32)
    hep.cms.label("Preliminary", lumi = label, data = True, loc=2, ax=axs[0], fontsize=18);
    hep.histplot(byHLTpath_vals, hist_pt_byHLTpath_wPS.axes['pt'].edges, ax = axs[2],histtype='fill')
    axs[2].set_yscale('log')
    axs[2].set_ylim([0.0, 1.0e10])
    axs[2].set_xlim([0.0, 2400])
    axs[2].set_title('Doubles Removed removed w/ prescale applied', pad=32)
    hep.cms.label("Preliminary", lumi = label, data = True, loc=2, ax=axs[2], fontsize=18);
    
    
    plt.rc('legend',fontsize='small')
    print(hist_pt.axes)
    axs[0].legend([cat for cat in hist_pt.project('HLT_cat').axes[0]])
    axs[1].legend([cat for cat in hist_pt_wPS.project('HLT_cat').axes[0]], ncol=2)
    axs[2].legend([cat for cat in hist_pt_byHLTpath_wPS.project('HLT_cat').axes[0]])
    dataset = 'JetHT' + str(year)
    os_path = 'plots/triggerStudies'+ dataset + '/'
    checkdir(os_path)
    plt.savefig(os_path + 'applyPrescales' + str(year) + ".png")


In [ ]:
with open("coffeaOutput/applyPrescalesL1_JetHT_2016_AK8PFJet.pkl", "rb") as f:
    result = pickle.load( f )
print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, '2016', label=r'Late 2016 - 16.8' )

In [ ]:
with open("coffeaOutput/applyPrescalesL1_JetHT_2017_AK8PFJet.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, '2017', label=r'2017 - 41.48',)

In [ ]:
with open("coffeaOutput/applyPrescalesL1_JetHT_2016APV_AK8PFJet.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, '2016APV', label=r'2016 APV - 19.5')

In [ ]:
with open("coffeaOutput/applyPrescalesL1_JetHT_2018_AK8PFJet.pkl", "rb") as f:
    result = pickle.load( f )
# print(result['hist_pt'][{'HLT_cat':sum, 'pt':sum}].values())
plotPrescales(result, '2018', label=r'2018 - 59.74',)